<a href="https://colab.research.google.com/github/YoungHyunKoo/GEE_Fall2025/blob/main/2_Explore_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **[GEO 6053] Remote Sensing - Fall 2025**
# **MODULE 2. Exploring Google Earth Engine (GEE)**

### OBJECTIVES
1. Explore  `ee` and `geemap` Python packages for mapping of remote sensing data
2. Import and visualize single-band and multi-band images from GEE
3. Export metadata information from the images

Credited by Younghyun Koo (kooala317@gmail.com)



## **1. Introduction to Google Earth Engine**

### What is Google Earth Engine?
**Google Earth Engine (GEE)** combines a multi-petabyte catalog of satellite imagery and geospatial datasets with planetary-scale analysis capabilities. Scientists, researchers, and developers use Earth Engine to detect changes, map trends, and quantify differences on the Earth's surface. Earth Engine is now available for commercial use, and remains free for academic and research use.



### Google Earth Engine Python API
The GEE was originally desined with JavaScript API (application programming interface), but Python API is also available. All of the GEE functions and data are provided through a Python library named `ee`. In this tutorial, we will also use another useful library named `geemap`, which is developed for a easier visualization of images for the GEE Python API. Both `ee` and `geemap` libraries are already installed in the Google Colab environment, so you can simply import them.
- `ee`: Earth Engine Python client library (https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)
- `geemap`: A Python package for interactive geospatial analysis and visualization with Google Earth Engine (https://geemap.org/)


<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/0*pzmEeRGOPHjpkfL6" width="700">

### Create Google Earth Engine cloud project
In order to start GEE, you should register your GEE cloud project with your Google account. Please go to the below website and complete the registration. You can select "Unpaid usage" and "Academia & Research" project.

https://earthengine.google.com/noncommercial/

## **2. Import `ee` and `geemap` libraries**

First of all, we should import `ee` library and initialize the API. When you initialize `ee`, please use the name of your own project.

In [ ]:
# Import ee library
import ee

try:
  # Initialize with your own project.
  ee.Initialize(project = "utsa-spring2024")
except:
  # Authenticate
  ee.Authenticate()
  # Initialize with your own project.
  ee.Initialize(project = "utsa-spring2024")

**🫡 NOTE:** The above `Authenticate` and `Initialize` should be run every time you start working on a new notebook.

If the above codes are run succefully, you are ready to use Earth Engine Python API. Congratulations!

Now, let's import `geemap` library. You can simply use the `import` command.

In [ ]:
# Import geemap library
import geemap

Let's draw a simple map with the `geemap` library.

In [ ]:
Map = geemap.Map() # Define a geemap map; Basemap options: basemap = "HYBRID", "TERRAIN", "SATELLITE", "ROADMAP"
Map

## **3. Load Image Collection**

### **Image Collection Overview**

Raster data are represented as `ee.Image` objects in Earth Engine. Images are composed of one or more **bands (channels)** and each band has its own name, data type, scale, mask and projection, etc. Each image also contains **metadata** stored as a set of properties. And an `ImageCollection` is a stack or sequence of these images. An ImageCollection can be loaded by pasting an GEE asset **ID** into the ImageCollection constructor. There are a bunch of various satellite images and geospatial datasets in GEE, and you can what datasets are available and description about those datasets through the [Google Earth Engine Data Catalogue](https://developers.google.com/earth-engine/datasets), where you can find the IDs of ImageCollection. In this tutorial, you will be able to how to handle these image collections, particularly Landsat and MODIS image collection, by temporal information (date and time), spatial information (latitude/longitude), and metadata (e.g. cloud covers).

<img src = "https://www.gislounge.com/wp-content/uploads/2020/10/earth-engine-data-catalog.png" width = "800">



### **3.1. Landsat image collection**

We will import [USGS Landsat 8 Collection 2 Tier 1 TOA (top-of-atmosphere) Reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA) image collection.

In [ ]:
# Import image collection - Landsat 8 surface reflectance
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

#### **Filter by dates**

There is a function named `filterDate` to filter an image collection by a date range. You need to provide start and end day for this function. You can find the details of `filterDate` function here: [filterDate](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate)

In [ ]:
# ImageCollection.filterDate(start, end)
collction = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate('2020-05-01', '2020-06-01')

**NOTE:** Start date is INCLUSIVE but end date is EXCLUSIVE.

#### **Filter by location**

In addition to the temporal filtering, you can use the `filterBounds` function to narrow down to your region of interest (ROI). You will filter out the image collection based on the GEE `geometry`. Before running the `filterBounds` function, you need to define the `geometry` as a ROI. We will set a ROI using `BBox` function. ([Link to BBox function](https://developers.google.com/earth-engine/apidocs/ee-geometry-bbox))

In [ ]:
# Region of interest as a bound box (longitude, latitude)
roi = ee.Geometry.BBox(-99, 29, -98, 30)
# San Antonio region: ee.Geometry.BBox(west, south, east, north)

In [ ]:
Map = geemap.Map()
Map.addLayer(roi, {}, "ROI")
Map.centerObject(roi, 8)
Map

In addition to the Point geometry, there are several other geometry types you can use: Line string, Linear ring, rectangle, polygon. Please visit the link below and practice how to create another type of geometry. [GEE geometry](https://developers.google.com/earth-engine/guides/geometries)

In [ ]:
# # Geometry
# point = ee.Geometry.Point([1.5, 1.5]);

# # Rectangle
# rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20]);

# # Polygon
# polygon = ee.Geometry.Polygon([
#   [[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]
# ]);

# # Line string
# lines = ee.Geometry.LineString([[5, -10], [35, -10], [35, 10], [5, 10], [5, -10]])

Now we will filter the image collection based on this defined geometry using the `filterBounds` function. Please find more details about this function here: [filterBounds](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterbounds)

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA") \
    .filterDate('2020-05-01', '2020-06-01') \
    .filterBounds(roi)

In [ ]:
# Number of images in the filtered image collection
print(collection.size().getInfo())

In [ ]:
# Load a geemap
Map = geemap.Map()

# "First" function gets the first image in the collection
image = collection.first()

# image visualization factors
vis_param = {'min': 0,
             'max': 0.2,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

Map.addLayer(image, vis_param, "First image")
Map.centerObject(image, 8)

Map

In [ ]:
# Check the image properties
geemap.image_props(image).getInfo()

#### **Filter by metadata**

As you can see in the image properties above, this image is from May 14, 2020. (See "system:time_end") However, the cloud cover of this image is about 22 % (i.e., 22 % of the entire image is covered by cloud). This cloud-covered image is not very useful for monitoring earth surfaces. Therefore, we need to select some images with low cloud covers. GEE provides the function named `filterMetadata` to filter out the image collection based on the metadata. To filter out cloud-covered images, we will use the 'CLOUD_COVER' property from the metadata. You can also find what properties this image collection have via this link: [Landsat TOA image properties](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA#image-properties)

In [ ]:
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA") \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterBounds(roi) \
    .filterMetadata('CLOUD_COVER', 'less_than', 10) \
    .sort("CLOUD_COVER")

# Filter by metadata: cloud cover is less than 10 %.
# In this case, we use 'less than' operator to filter out low cloud cover area.
# However, there are some other operators as well: 'greater than', 'equals', etc.
# .sort function is to sort the images in the image collection based on cloud covers in ascending order.
# That is, the images are sorted from low cloud covers to higher cloud covers

In [ ]:
# Number of images in the filtered image collection
print(collection.size().getInfo())

In [ ]:
# Load a geemap
Map = geemap.Map()

# "First" function gets the first image in the collection
image = collection.first()

# image visualization factors
vis_param = {'min': 0,
             'max': 0.3,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

Map.addLayer(image, vis_param, "First image")
Map.centerObject(image, 8)

Map

In [ ]:
geemap.image_props(image).getInfo()

You can extract the information of the images in the image collection as a form of array by using "aggregate_array" function.

In [ ]:
# Cloud cover information as a array
collection.aggregate_array('CLOUD_COVER').getInfo()

In [ ]:
# Image ID information as a array
collection.aggregate_array('system:id').getInfo()

In [ ]:
# From this image ID information, you can import an image you want from the image collection.

# Array of IDs is saved as a variable named "id_array"
id_array = collection.aggregate_array('system:id').getInfo()

# Import the 3rd image (index: 3) as a image
image = ee.Image(id_array[3])

# image visualization factors
vis_param = {'min': 0,
             'max': 0.3,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

Map.addLayer(image, vis_param, "First image")
Map.centerObject(image, 8)

Map

The image we just draw on the map is a **true color** image. True color image means that it is visualized by the way we normally see with our eyes. Every color we see is determined by the composition of three components of light (R, G, B). In a true color image, red band is assigned to red component, green band is assigned to green component, and blue band is assigned to blue component; this is the exactly same to what we see with our eyes. However, we can also create a **false color** image. For example, if we assign near-infrared band (0.85-0.88 μm; band 5) to red component, red band (0.64-0.67 μm; band 4) to green component, and green band (0.53-0.59 μm; band 3) to blue component, then this false color image is pretty diffent with what we see in the real world. We can use this false color image to highlight some features (e.g. vegetation, water, etc.)

In [ ]:
# Create a new geemap map
Map = geemap.Map()

Map.centerObject(image, zoom=8)

# Infrared false color image (i.e. R = Band 5, G = Band 4, B = Band 3)
vis_params = {'bands': ['B5', 'B4', 'B3'], 'min': 0.0, 'max': 0.3, 'gamma': 1.0}
Map.addLayer(image, vis_params, "Landsat False color")

Map

***DO IT YOURSELF!!***
- Try differnt composition of the RGB component for the false color image. In your opinion, what kind of false color composition is useful to distinguish urban and forest area?

### **4.1. MODIS image collection**

Next, we will import another image collection for MODIS data: [MODIS Terra Surface Reflectance Daily Global 1km and 500m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09GA)

In [ ]:
# Import image collection - Landsat 8 surface reflectance
collection = ee.ImageCollection("MODIS/061/MOD09GA")

#### **Filter image collection**



Similar to the Landsat example, we will filter the MODIS image collection by time. FYI, this daily MODIS surface reflectance image collection is a daily "global" product (i.e., daily mosaic of the entire Earth surface) and does not contain image properties as metadata.

In [ ]:
collection = ee.ImageCollection("MODIS/061/MOD09GA").filterDate('2020-05-01', '2020-05-31')

In [ ]:
# Number of images in the filtered image collection
print(collection.size().getInfo())

In [ ]:
# Load a geemap
Map = geemap.Map()

# "First" function gets the first image in the collection
image = collection.first() #.first()

# image visualization factors: RGB = 143
vis_param = {'min': 0,
             'max': 8000,
             'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
             'gamma': 1.0}

Map.addLayer(image, vis_param, "MODIS")
Map.centerObject(roi, 8)

Map

## **References**
- https://geemap.org/tutorials/
- https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api